In [ ]:
!git clone https://github.com/tGhattas/bamba.git

In [ ]:
%cd bamba

In [ ]:
!git pull
!pip install . -q

In [ ]:
!nvidia-smi

In [ ]:
!pwd

In [ ]:
# prompt: authenticate hugging face

!pip install huggingface_hub -q

from huggingface_hub import notebook_login

notebook_login()


In [ ]:
!pip3 install causal-conv1d>=1.2.0 -q
!pip3 install datasets -q
# !pip install . -q
# !pip install zstandard -q
!pip3 install torch -q
!pip3 install tqdm -q
!pip3 install wandb -q
!pip3 install transformers #-q



In [ ]:
#wandb login
!pip3 install --upgrade transformers -q


In [ ]:
import importlib
import  evals.distillation as dm
# importlib.reload(dm)

In [ ]:
import torch
"cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
dm.train(limit=5000)

In [1]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from itertools import islice
# from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel, MambaConfig
from tqdm import tqdm
HF_PADDING_IGNORE = -100

# teacher_model_path = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"


/Users/tamer/miniconda3/envs/myvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = load_dataset("monology/pile-uncopyrighted", streaming=True)


# Load the teacher tokenizer
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_path)

# Tokenize the dataset
def tokenize_function(examples):
    return teacher_tokenizer(examples["text"], truncation=True, padding="max_length", max_length=1024, return_tensors="pt")

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Create the data loader
data_loader = DataLoader(tokenized_datasets["train"], batch_size=4, num_workers=4)


In [ ]:
for i in tokenized_datasets["train"]:
    print(i)
    break

In [ ]:
for i in tokenized_datasets["train"]:
    print(teacher_tokenizer.decode(i["input_ids"]))
    break

In [ ]:
from itertools import islice

for i in islice(tokenized_datasets["train"],50):
    print(i["input_ids"])
    print(i["input_ids"].shape)
    print(i["attention_mask"])
    

In [ ]:
data_loader = DataLoader(tokenized_datasets["train"], batch_size=4, num_workers=4)

In [ ]:
for batch in islice(data_loader, 10):
    print(batch)
    break

In [27]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig



teacher_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
student_tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-790m-hf")
teacher_vocab = set(teacher_tokenizer.get_vocab())
student_vocab = set(student_tokenizer.get_vocab())
student_tokenizer.add_tokens(list(teacher_vocab - student_vocab))
tokens_to_remove = list(student_vocab - teacher_vocab)
new_student_vocab = student_tokenizer.get_vocab()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
class ModifiedMambaTokenizerFactory:
    def __init__(self, student_tokenizer: AutoTokenizer, teacher_tokenizer: AutoTokenizer):
        self.student_tokenizer = student_tokenizer
        self.teacher_tokenizer = teacher_tokenizer
        teacher_vocab = set(teacher_tokenizer.get_vocab())
        student_vocab = set(student_tokenizer.get_vocab())
        student_tokenizer.add_tokens(list(teacher_vocab - student_vocab)[:len(teacher_vocab) - len(student_vocab)]) #
    

    def get_modified_tokenizer(self):
        return self.student_tokenizer

In [3]:
teacher_tokenizer0 = AutoTokenizer.from_pretrained("EleutherAI/pythia-14m")
teacher_tokenizer4 = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
teacher_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1b")
teacher_tokenizer2 = AutoTokenizer.from_pretrained("EleutherAI/pythia-1.4b")
student_tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-790m-hf")
student_tokenizer2 = AutoTokenizer.from_pretrained("state-spaces/mamba-370m-hf")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
len(student_tokenizer.get_vocab()), len(student_tokenizer2.get_vocab()),  len(teacher_tokenizer.get_vocab()), len(teacher_tokenizer2), len(teacher_tokenizer0), len(teacher_tokenizer4)

(50277, 50277, 50277, 50277, 50277, 32000)

In [15]:
tmodel = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-1b")
smodel = AutoModelForCausalLM.from_pretrained("state-spaces/mamba-790m-hf")
smodel2 = AutoModelForCausalLM.from_pretrained("state-spaces/mamba-370m-hf")
tmodelw = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")


In [16]:
tmodel.config.vocab_size, smodel.config.vocab_size, smodel2.config.vocab_size, tmodelw.config.vocab_size

(50304, 50280, 50280, 32000)

In [11]:
smodel2.resize_token_embeddings(len(student_tokenizer2))
tmodel.resize_token_embeddings(len(student_tokenizer2))

Embedding(50277, 2048)

In [10]:
smodel2

MambaForCausalLM(
  (backbone): MambaModel(
    (embeddings): Embedding(50277, 1024)
    (layers): ModuleList(
      (0-47): 48 x MambaBlock(
        (norm): MambaRMSNorm()
        (mixer): MambaMixer(
          (conv1d): Conv1d(2048, 2048, kernel_size=(4,), stride=(1,), padding=(3,), groups=2048)
          (act): SiLU()
          (in_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (x_proj): Linear(in_features=2048, out_features=96, bias=False)
          (dt_proj): Linear(in_features=64, out_features=2048, bias=True)
          (out_proj): Linear(in_features=2048, out_features=1024, bias=False)
        )
      )
    )
    (norm_f): MambaRMSNorm()
  )
  (lm_head): Linear(in_features=1024, out_features=50277, bias=False)
)

In [8]:
tmodel

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-15): 16 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True

In [12]:
tmodel

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50277, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-15): 16 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True

In [ ]:
print(len(teacher_tokenizer.get_vocab()), len(student_tokenizer.get_vocab()))
print(len(tokens_to_remove))
print(len(new_student_vocab))


In [ ]:
new_vocab = {k: v for k, v in sorted(new_student_vocab.items(), key=lambda item: item[1])}